In [ ]:
#Connect to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Packages
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
import nltk
from nltk.tokenize import sent_tokenize
import re
import json
import time

In [ ]:
def bing_df(url, num_pages=5):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    search_results = []

    for page in range(num_pages):
        paginated_url = f"{url}&first={page * 10 + 1}"
        response = requests.get(paginated_url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            result_containers = soup.find_all('li', {'class': 'b_algo'})

            for result in result_containers:
                title_tag = result.find('h2')
                title = title_tag.text if title_tag else ''

                link_tag = result.find('a')
                link = link_tag['href'] if link_tag and 'href' in link_tag.attrs else ''

                description_tag = result.find('p')
                description = description_tag.text if description_tag else ''

                displayed_link_tag = result.find('cite')
                displayed_link = displayed_link_tag.text if displayed_link_tag else ''

                snippet_tag = result.find('div', {'class': 'b_caption'})
                snippet = snippet_tag.text if snippet_tag else ''

                additional_info_tag = result.find('div', {'class': 'b_attribution'})
                additional_info = additional_info_tag.text if additional_info_tag else ''

                publish_time_tag = result.find('span', {'class': 'news_dt'})
                publish_time = publish_time_tag.text if publish_time_tag else ''

                search_results.append({
                    'Title': title,
                    'Link': link,
                    'Description': description,
                    'Displayed Link': displayed_link,
                    'Snippet': snippet,
                    'Additional Info': additional_info,
                    'Publish Time': publish_time
                })
        else:
            print(f"Failed to retrieve page {page + 1}. Status code: {response.status_code}")
            break

    return pd.DataFrame(search_results)


In [ ]:
def fetch_html(url):
    try:
        response = requests.get(url, timeout=15)
        response.raise_for_status()
        #print("flag")
        return response.text
    except requests.exceptions.RequestException as e:
        return None

In [ ]:
#Input
NJ_areas=pd.read_csv("/content/drive/MyDrive/PSEG Research Project/New Jersey Areas/NJ_with_lat_long.csv")
NJ_areas.head()
NJ_areas['Bing Search Link'] = NJ_areas['Bing Search Link'].apply(lambda x: x.replace(" ", "+"))

In [ ]:
NJ_areas

,Municipality,County,MRI Score,Latitude,Longitude,Bing Search Link
0,Aberdeen township,Monmouth,2.33,40.407114,-74.221629,https://www.bing.com/search?q=Aberdeen+townshi...
1,Absecon city,Atlantic,-2.94,39.436241,-74.517871,https://www.bing.com/search?q=Absecon+city+New...
2,Alexandria township,Hunterdon,3.95,40.591417,-75.012824,https://www.bing.com/search?q=Alexandria+towns...
3,Allamuchy township,Warren,4.03,40.921765,-74.810166,https://www.bing.com/search?q=Allamuchy+townsh...
4,Alloway township,Salem,0.83,39.560857,-75.362397,https://www.bing.com/search?q=Alloway+township...
...,...,...,...,...,...,...
348,Woodbury city,Gloucester,-6.38,39.845233,-75.147237,https://www.bing.com/search?q=Woodbury+city+Ne...
349,Woodland township,Burlington,-3.83,39.842075,-74.523058,https://www.bing.com/search?q=Woodland+townshi...
350,Woodstown borough,Salem,-1.92,39.651287,-75.329138,https://www.bing.com/search?q=Woodstown+boroug...
351,Woolwich township,Gloucester,2.89,39.741482,-75.348055,https://www.bing.com/search?q=Woolwich+townshi...


In [ ]:
for i in range(len(NJ_areas)):
  url=NJ_areas['Bing Search Link'][i]
  name=NJ_areas['Municipality'][i]
  #time.sleep(10)
  print(f'df = bing_df("{url}", 5)')
  print(f"df.to_csv('/content/drive/MyDrive/PSEG Research Project/New Jersey Areas/Datasets/bing/{name}.csv')")
  #df = bing_df(url, 5)


df = bing_df("https://www.bing.com/search?q=Aberdeen+township+New+Jersey", 5)
df.to_csv('/content/drive/MyDrive/PSEG Research Project/New Jersey Areas/Datasets/bing/Aberdeen township.csv')
df = bing_df("https://www.bing.com/search?q=Absecon+city+New+Jersey", 5)
df.to_csv('/content/drive/MyDrive/PSEG Research Project/New Jersey Areas/Datasets/bing/Absecon city.csv')
df = bing_df("https://www.bing.com/search?q=Alexandria+township+New+Jersey", 5)
df.to_csv('/content/drive/MyDrive/PSEG Research Project/New Jersey Areas/Datasets/bing/Alexandria township.csv')
df = bing_df("https://www.bing.com/search?q=Allamuchy+township+New+Jersey", 5)
df.to_csv('/content/drive/MyDrive/PSEG Research Project/New Jersey Areas/Datasets/bing/Allamuchy township.csv')
df = bing_df("https://www.bing.com/search?q=Alloway+township+New+Jersey", 5)
df.to_csv('/content/drive/MyDrive/PSEG Research Project/New Jersey Areas/Datasets/bing/Alloway township.csv')
df = bing_df("https://www.bing.com/search?q=Andover+borough+N

In [ ]:
df = bing_df(bing_url, num_pages)
df['Inner HTML'] = df['Displayed Link'].apply(fetch_html)
df = df.dropna(subset=['Inner HTML'])
df=df.reset_index()
df=df.drop(columns=['index'])